# Model

This notebook loads your assembled and cleaned data and runs modeling and analysis on it.

- First, we run predictive models
- Then, we analyze your predictions as well as existing ones (such as the assessor's)
- Finally, we generate automated statistical reports assessing their quality

In [ ]:
# Change these as desired

# The slug of the locality you are currently working on
locality = "us-md-baltimorecity"

# Whether to print out a lot of stuff (can help with debugging) or stay mostly quiet
verbose = True

# Clear previous state for this notebook and start fresh
clear_checkpoints = True

# Clear cached results for model predictions and start fresh
clear_model_results = True

# 1. Basic setup

In [ ]:
import init_notebooks
init_notebooks.setup_environment()

In [ ]:
# import a bunch of stuff
from openavmkit.pipeline import (
    init_notebook,
    load_settings,
    read_pickle,
    examine_sup,
    write_canonical_splits,
    try_variables,
    try_models,
    finalize_models,
    run_and_write_ratio_study_breakdowns,
    enrich_sup_spatial_lag_sale_price
    from_checkpoint,
    delete_checkpoints
)

In [ ]:
init_notebook(locality)

In [ ]:
if clear_checkpoints:
    delete_checkpoints("3-model")

In [ ]:
settings = load_settings()

# 2. Prepare

We load the cleaned data from the last checkpoint:

In [ ]:
# load the data
sales_univ_pair = read_pickle("out/2-clean-sup")

We separate our test set from our training set.  
This guarantees we have one durable source of truth for test/train set.

In [ ]:
write_canonical_splits(
    sales_univ_pair,
    load_settings()
)

In [ ]:
sales_univ_pair = from_checkpoint("3-model-00-enrich-spatial-lag", enrich_sup_spatial_lag,
    {
        "sup": sales_univ_pair,
        "settings": settings,
        "verbose": verbose
    }
)

We enrich with spatial lag

In [ ]:
examine_sup(sales_univ_pair, settings)

# 3. Experiment

Try out variables and models before running the real thing

In [ ]:
try_variables(
    sales_univ_pair,
    load_settings(),
    verbose,
    plot = False
)

In [ ]:
try_models(
    sup=sales_univ_pair,
    settings=load_settings(),
    save_params=True,
    verbose=verbose,
    run_main=True,
    run_vacant=True,
    run_hedonic=True
)

# 4. Finalize models

Once we've locked in good values, we finalize our results

In [ ]:
results = from_checkpoint("3-model-02-finalize-models", finalize_models,
    {
        "sup": sales_univ_pair,
        "settings": load_settings(),
        "save_params": True,
        "use_saved_params": True,
        "verbose": verbose
    }
)

# 5. Generate reports

In [ ]:
# run ratio study reports
run_and_write_ratio_study_breakdowns(load_settings())

# 6. Make it map ready